In [4]:
import requests
import pandas as pd
from config import tmdb_key

In [5]:
#setting up params
base_params = dict(api_key=tmdb_key)
year = 2018
page = 1
listing_params = dict(primary_release_year=year,region='US', sort_by='revenue.desc', include_adult='false', language='en-US', page='1')
listing_params.update(base_params)
response = requests.get('https://api.themoviedb.org/3/discover/movie', params=listing_params)
imdb_films = response.json()['results']


In [6]:
#This takes approx 10-15 to run if you wish to cut the sample size change where it says MODIFY to for example "page == 2"
rows = []

restart = True
page_limit = True
while restart == True:
   while page_limit == True:
      for film in imdb_films:
         film_revenue = requests.get('https://api.themoviedb.org/3/movie/{}'.format(film['id']), params=listing_params)
         film_revenue = film_revenue.json()
         try:
            row = dict(title=film['title'], revenue=film_revenue['revenue'], Year=listing_params['primary_release_year'],\
                     VoteAVG=film['vote_average'], VoteCT=film['vote_count'], Popularity=film['popularity'], Date=film['release_date'],\
                        Genre=film['genre_ids'][0])
         except:
            pass   
         rows.append(row)
      #MODIFY page == x: to increase or decrease sample size
      if page == 10: 
         page_limit = False
      else:
         page += 1
         listing_params.update({'page':page})
         response = requests.get('https://api.themoviedb.org/3/discover/movie', params=listing_params)
         imdb_films = response.json()['results']
   if year == 2023: 
      restart = False
   else:
      page_limit = True
      page = 1
      year += 1
      listing_params.update({'primary_release_year':year})
      listing_params.update({'page':page})
      response = requests.get('https://api.themoviedb.org/3/discover/movie', params=listing_params)
      imdb_films = response.json()['results']
      
df = pd.DataFrame(rows, columns=['title', 'revenue', 'Year','VoteAVG', 'VoteCT', 'Popularity', 'Date', 'Genre'])

In [7]:
#copy df to avoid any issues
df_copy = df.copy()

In [8]:
#replace genre codes with recognisabled genre tags
data = pd.DataFrame(
       {
              'A': [28,12,16,35,80,99,18,10751,14,36,27,10402,9648,10749,878,10770,53,10752,37],
              'B':['Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family','Fantasy','History',\
                     'Horror','Music','Mystery','Romance','Science', 'Fiction','TV Movie','War','Western']
       }
)
map_dict = {28:'Action', 12:'Adventure'}
map_dict = {x:data['B'].tolist()[i] for i,x in enumerate(data['A'].tolist())}
df_copy['Genre'] = df_copy['Genre'].map(map_dict)
df_copy.head()

,title,revenue,Year,VoteAVG,VoteCT,Popularity,Date,Genre
0,Avengers: Infinity War,2052415039,2018,8.3,27946,197.252,2018-04-27,Adventure
1,Black Panther,1349926083,2018,7.4,21169,62.546,2018-02-16,Action
2,Jurassic World: Fallen Kingdom,1310466296,2018,6.5,10525,57.672,2018-06-22,Action
3,Incredibles 2,1242805359,2018,7.5,12140,78.707,2018-06-15,Action
4,Aquaman,1148528393,2018,6.9,13069,54.338,2018-12-21,Action


In [9]:
#export df_copy to a csv to be read in film_analysis
df_copy.to_csv("output_data/tmdb_movies.csv", index_label="Index")